In [1]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 2.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
import joblib

In [3]:
np. __version__ 

'1.22.4'

In [4]:
URL_data_sme_minmax_be = 'https://drive.google.com/file/d/1JV61ruhmwiLtUDsG6VdSLI_EdtlNvAuC/view?usp=share'
path_data_sme_minmax_be = 'https://drive.google.com/uc?export=download&id='+URL_data_sme_minmax_be.split('/')[-2]
data_sme_minmax_be = pd.read_csv(path_data_sme_minmax_be)

URL_data_sme_standard_be = 'https://drive.google.com/file/d/1kApVPeAqR4EPAp2Qqe5SGVcv3g13PjGE/view?usp=share'
path_data_sme_standard_be = 'https://drive.google.com/uc?export=download&id='+URL_data_sme_standard_be.split('/')[-2]
data_sme_standard_be = pd.read_csv(path_data_sme_standard_be)

URL_data_test_minmax_be = 'https://drive.google.com/file/d/1xzEL_joTrtUhRnUfu9x_5j7bgZJNlxqD/view?usp=share'
path_data_test_minmax_be = 'https://drive.google.com/uc?export=download&id='+URL_data_test_minmax_be.split('/')[-2]
data_test_minmax_be = pd.read_csv(path_data_test_minmax_be)

URL_data_test_standard_be = 'https://drive.google.com/file/d/19rJwtDQincWlBWw4pTS6r8Cvf11aKDfW/view?usp=share'
path_data_test_standard_be = 'https://drive.google.com/uc?export=download&id='+URL_data_test_standard_be.split('/')[-2]
data_test_standard_be = pd.read_csv(path_data_test_standard_be)


In [5]:
x_test_minmax_be = data_test_minmax_be.drop('level', axis=1)
y_test_minmax_be = data_test_minmax_be['level']

x_test_standard_be = data_test_standard_be.drop('level', axis=1)
y_test_standard_be = data_test_standard_be['level']

In [10]:
def RandomForest(x_train, y_train, x_test, y_test):
    rf = RandomForestClassifier()

    # Define the hyperparameter search space
    n_estimators = (10, 100)
    max_features = ['auto', 'sqrt']
    max_depth = (10, 110)
    min_samples_split = (2, 10)
    min_samples_leaf = (1, 4)
    bootstrap = [True, False]

    search_space = {
        'n_estimators': n_estimators,
        'max_features': max_features,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf': min_samples_leaf,
        'bootstrap': bootstrap
    }

    # Define the hyperparameter search method
    rf_search = BayesSearchCV(
        rf, search_space, n_iter=50, cv=5, n_jobs=-1, random_state=42
    )

    rf_search.fit(x_train, y_train)
    # Select the hyperparameters that result in the best performance
    best_params = rf_search.best_params_

    print("Best parameters: ", best_params)

    rf = RandomForestClassifier(**best_params)

    rf.fit(x_train, y_train)

    # predict data train
    y_pred_train_labels = rf.predict(x_train)

    # Calculate the F1 score, recall, precision, and accuracy
    f1_train = f1_score(y_train, y_pred_train_labels, average='macro')
    recall_train = recall_score(y_train, y_pred_train_labels, average='macro')
    precision_train = precision_score(y_train, y_pred_train_labels, average='macro')
    accuracy_train = accuracy_score(y_train, y_pred_train_labels)
    
    print("\nEvaluation data training: ")
    print("Macro-Averaged Recall: ", f1_train)
    print("Macro-Averaged Precision: ", recall_train)
    print("Macro-Averaged Precision: ", precision_train)
    print("Accuracy: ", accuracy_train)

    # predict data test
    y_pred_labels = rf.predict(x_test)

    # Calculate the macro-averaged F1 score, recall, precision, and accuracy
    f1 = f1_score(y_test, y_pred_labels, average='macro')
    recall = recall_score(y_test, y_pred_labels, average='macro')
    precision = precision_score(y_test, y_pred_labels, average='macro')
    accuracy = accuracy_score(y_test, y_pred_labels)

    print("\nEvaluation data training: ")
    print("Macro-Averaged F1 Score: ", f1)
    print("Macro-Averaged Recall: ", recall)
    print("Macro-Averaged Precision: ", precision)
    print("Accuracy: ", accuracy)

    joblib.dump(rf, 'model_rf1.joblib')

## Data dengan kombinasi SMOTE-ENN

Data dengan MinMaxScaler

In [7]:
x_train_sme_minmax_be = data_sme_minmax_be.drop('level', axis=1)
x_train_sme_minmax_be.sample(6)

,precip,Libur,ROAD_CLOSED,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,...,street_1,street_2,street_3,street_4,street_5,street_6,street_7,street_8,street_9,street_10
71880,0.000017,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,0.000000,1.0,0.0,1.0,0.000000,0.0,1.0,0.000000
122521,0.000245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.000000,0.194924,0.0,1.0,0.0,0.000000,0.0,0.0,0.805076
99313,0.004022,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.039598,1.000000,0.0,0.0,0.0,0.039598,0.0,1.0,0.000000
18314,0.000040,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.000000,1.000000,1.0,0.0,1.0,0.000000,0.0,0.0,0.000000
79868,0.079130,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,1.000000,0.0,0.0,1.0,1.000000,1.0,1.0,1.000000
16077,0.060046,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.000000,1.000000,0.0,0.0,0.0,1.000000,1.0,0.0,0.000000


In [8]:
y_train_sme_minmax_be = data_sme_minmax_be['level']

In [11]:
RandomForest(x_train_sme_minmax_be, y_train_sme_minmax_be, x_test_minmax_be, y_test_minmax_be)

/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters:  OrderedDict([('bootstrap', False), ('max_depth', 38), ('max_features', 'sqrt'), ('min_samples_leaf', 1), ('min_samples_split', 2), ('n_estimators', 100)])

Evaluation data training: 
Macro-Averaged Recall:  1.0
Macro-Averaged Precision:  1.0
Macro-Averaged Precision:  1.0
Accuracy:  1.0

Evaluation data training: 
Macro-Averaged F1 Score:  0.6137636931087993
Macro-Averaged Recall:  0.6252684644486283
Macro-Averaged Precision:  0.6036733541663667
Accuracy:  0.7826186027434717


In [12]:
model_rf_sme_minmax = joblib.load('model_rf1.joblib')

In [13]:
joblib.dump(model_rf_sme_minmax, 'model_rf_sme_minmax.joblib')

['model_rf_sme_minmax.joblib']

Data dengan StandardScaler

In [14]:
x_train_sme_standard_be = data_sme_standard_be.drop('level', axis=1)
x_train_sme_standard_be.sample(6)

,precip,Libur,ROAD_CLOSED,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,...,street_1,street_2,street_3,street_4,street_5,street_6,street_7,street_8,street_9,street_10
141494,-0.267025,-0.223265,14.951970,2.444868,-0.405794,-0.40954,-0.409085,-0.407522,-0.408532,-0.408239,...,-0.890382,-0.886265,1.105498,1.007528,1.004424,-0.993110,1.004149,-0.998166,0.996724,-1.003757
151723,-0.261183,-0.223265,14.951970,-0.409020,-0.405794,-0.40954,-0.409085,-0.407522,-0.408532,2.449546,...,-0.890382,-0.886265,-0.904570,-0.992528,1.004424,-0.993110,-0.995868,-0.998166,0.996724,-1.003757
163326,-0.148788,-0.223265,14.951970,-0.409020,-0.405794,-0.40954,2.444479,-0.407522,-0.408532,-0.408239,...,-0.890382,-0.886265,-0.904570,1.007528,1.004424,1.006938,-0.995868,-0.998166,-1.003287,-1.003757
37981,0.259280,-0.223265,-0.066881,-0.409020,-0.405794,-0.40954,2.444479,-0.407522,-0.408532,-0.408239,...,-0.890382,-0.886265,-0.904570,-0.992528,0.985495,-0.993110,-0.995868,1.001838,0.996724,-1.003757
102100,-0.205473,-0.223265,-0.066881,2.444868,-0.405794,-0.40954,-0.409085,-0.407522,-0.408532,-0.408239,...,-0.890382,-0.886265,-0.904570,1.007528,1.004424,1.006938,1.004149,-0.998166,0.996724,-1.003757
116825,7.446791,-0.223265,-0.066881,-0.409020,-0.405794,-0.40954,-0.409085,-0.407522,2.447789,-0.408239,...,-0.085491,1.128331,1.105498,-0.992528,-0.995596,1.006938,1.004149,0.202340,0.197223,0.996257


In [15]:
y_train_sme_standard_be = data_sme_standard_be['level']

In [16]:
RandomForest(x_train_sme_standard_be, y_train_sme_standard_be, x_test_standard_be, y_test_standard_be)

/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters:  OrderedDict([('bootstrap', False), ('max_depth', 110), ('max_features', 'sqrt'), ('min_samples_leaf', 1), ('min_samples_split', 2), ('n_estimators', 100)])

Evaluation data training: 
Macro-Averaged Recall:  1.0
Macro-Averaged Precision:  1.0
Macro-Averaged Precision:  1.0
Accuracy:  1.0

Evaluation data training: 
Macro-Averaged F1 Score:  0.6174431632924514
Macro-Averaged Recall:  0.6286376635888155
Macro-Averaged Precision:  0.6077538096816845
Accuracy:  0.783074328943171


In [17]:
model_rf_sme_standard = joblib.load('model_rf1.joblib')

In [18]:
joblib.dump(model_rf_sme_standard, 'model_rf_sme_standard.joblib')

['model_rf_sme_standard.joblib']